In [ ]:
#Notebook to check the results of functions tidalhl.py 

In [ ]:
import pandas as pd

In [ ]:
import hvplot.pandas

In [ ]:
from pydsm.functions import tidalhl

In [ ]:
df=pd.read_csv('../tests/data/tidal_signal_with_gaps.csv',index_col=0,parse_dates=[0])
df.index.freq=pd.infer_freq(df.index)

In [ ]:
df.hvplot.table()

In [ ]:
df.hvplot()

In [ ]:
dfh,dfl=tidalhl.get_tidal_hl_rolling(df)
dfamp=tidalhl.get_tidal_amplitude(dfh,dfl)

In [ ]:
df.hvplot(label='original')*dfamp.hvplot.scatter(label='amplitude')

In [ ]:
pd.concat([dfh,dfl,dfamp],axis=1)

In [ ]:
dfh.hvplot.scatter(label='highs')*dfl.hvplot.scatter(label='lows',marker='o')*df.hvplot(label='data')*dfamp.hvplot(label='amp')

In [ ]:
df2=2*df # increase amplitude to twice
df2s=df2.resample('T').interpolate().shift(25,freq='T') # resample @ min resolution and shift by 25 mins
df2s=df2s.resample(df.index.freq).interpolate() # resample back at original resolution 

In [ ]:
df2s.hvplot(label='shifted')*df2.hvplot(label='data')

In [ ]:
df2.hvplot(label='doubled')*df.hvplot(label='original')

In [ ]:
dfamp2=tidalhl.get_tidal_amplitude(*tidalhl.get_tidal_hl_rolling(df2))
display(dfamp,dfamp2)

In [ ]:
dfampdiff=tidalhl.get_tidal_amplitude_diff(dfamp,dfamp2)
dfampdiff

In [ ]:
dfamp2s=tidalhl.get_tidal_amplitude(*tidalhl.get_tidal_hl_rolling(df2s))
display(dfamp,dfamp2s)

In [ ]:
dfampdiffs=tidalhl.get_tidal_amplitude_diff(dfamp,dfamp2s)
dfampdiffs

In [ ]:
def get_tidal_amplitude_diff(dfamp1, dfamp2, percent_diff=False):
    '''
    Get the difference of values within +/- 4H of values in the two amplitude arrays
    '''
    dfamp = pd.concat([dfamp1, dfamp2], axis=1).dropna(how='all')
    dfamp.columns = ['2', '1']
    tdelta = '4H'
    sliceamp = [slice(t-pd.to_timedelta(tdelta), t+pd.to_timedelta(tdelta)) for t in dfamp.index]
    ampdiff = [get_value_diff(dfamp[sl], percent_diff) for sl in sliceamp]
    return ampdiff

In [ ]:
dfa=pd.concat([dfamp,dfamp2s],axis=1).dropna(how='all')

In [ ]:
dfa

In [ ]:
dfa.columns=['1','2']
dfa.hvplot.scatter()

In [ ]:
tdelta = '4H'
sliceamp = [slice(t-pd.to_timedelta(tdelta), t+pd.to_timedelta(tdelta)) for t in dfa.index]
ampdiff = [(sl.start, tidalhl.get_value_diff(dfa[sl], percent_diff=False)) for sl in sliceamp]
ampdiff

In [ ]:
dfa

In [ ]:
dfa[sliceamp[0]]

In [ ]:
tidalhl.get_value_diff(dfa[sliceamp[0]])

In [ ]:
tidalhl.get_index_diff(dfa[sliceamp[0]])

In [ ]:
pd.DataFrame(ampdiff,dfa.index)

In [ ]:
sliceamp

In [ ]:
tidalhl.get_tidal_phase_diff(*tidalhl.get_tidal_hl_rolling(df),*tidalhl.get_tidal_hl_rolling(df2s))